In [31]:
!pip3 install torch torchvision numpy


[notice] A new release of pip is available: 24.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


### Game 2048 Environment and DQN Agent

In [ ]:
import numpy as np
import random
from collections import deque
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

class Game2048Env:
    def __init__(self):
        self.size = 4
        self.reset()

    def reset(self):
        self.board = np.zeros((4, 4), dtype=int)
        self._add_tile()
        self._add_tile()
        self.score = 0
        self.done = False
        return self.board.copy()

    def _add_tile(self):
        empty = list(zip(*np.where(self.board == 0)))
        if not empty:
            return
        i, j = random.choice(empty)
        self.board[i][j] = 2 if random.random() < 0.9 else 4

    def _move_line(self, line):
        new_line = [i for i in line if i != 0]
        score_increase = 0
        merged_line = []
        skip = False
        for i in range(len(new_line)):
            if skip:
                skip = False
                continue
            if i + 1 < len(new_line) and new_line[i] == new_line[i + 1]:
                merged_value = new_line[i] * 2
                merged_line.append(merged_value)
                score_increase += merged_value
                skip = True
            else:
                merged_line.append(new_line[i])
        merged_line += [0] * (self.size - len(merged_line))
        return np.array(merged_line), score_increase

    def _move_board(self, direction):
        board = self.board.copy()
        total_reward = 0
        rotated_board = np.rot90(board, k=direction)
        new_board = np.zeros_like(rotated_board)
        for i in range(self.size):
            line = rotated_board[i, :]
            new_line, reward = self._move_line(line)
            new_board[i, :] = new_line
            total_reward += reward
        final_board = np.rot90(new_board, k=-direction)
        return final_board, total_reward

    def step(self, action):
        if self.done:
            return self.board.copy(), 0, True, {}
        new_board, reward = self._move_board(action)
        if not np.array_equal(new_board, self.board):
            self.board = new_board
            self._add_tile()
            self.score += reward
            if self.is_done():
                self.done = True
        else:
            reward = -10
        return self.board.copy(), reward, self.done, {}
    
    def _move(self, board, action):
        rotated_board = np.rot90(board, k=action)
        new_board = np.zeros_like(rotated_board)
        for i in range(self.size):
            line = rotated_board[i, :]
            new_line, reward = self._move_line(line)
            new_board[i, :] = new_line
        final_board = np.rot90(new_board, k=-action)
        return final_board
    
    def get_valid_actions(self):
        valid_actions = []
        for action in range(4):
            temp_board = self.board.copy()
             # Apply the move logic manually
            rotated_board = np.rot90(temp_board, k=action)
            new_board = np.zeros_like(rotated_board)
            for i in range(self.size):
                line = rotated_board[i, :]
                new_line, _ = self._move_line(line)
                new_board[i, :] = new_line
            final_board = np.rot90(new_board, k=-action)
            
            if not np.array_equal(final_board, temp_board):
                valid_actions.append(action)
        
        return valid_actions

    def is_done(self):
        return len(self.get_valid_actions()) == 0


In [5]:
import numpy as np
import random
from collections import deque
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# --- Optimized Environment ---
class Game2048Env:
    def __init__(self):
        self.size = 4
        self.reset()

    def reset(self):
        self.board = np.zeros((4, 4), dtype=int)
        self._add_tile()
        self._add_tile()
        self.score = 0
        self.done = False
        return self.board.copy()

    def _add_tile(self):
        empty = list(zip(*np.where(self.board == 0)))
        if not empty:
            return
        i, j = random.choice(empty)
        self.board[i][j] = 2 if random.random() < 0.9 else 4

    def _move_line(self, line):
        new_line = [i for i in line if i != 0]
        score_increase = 0
        merged_line = []
        skip = False
        for i in range(len(new_line)):
            if skip:
                skip = False
                continue
            if i + 1 < len(new_line) and new_line[i] == new_line[i + 1]:
                merged_value = new_line[i] * 2
                merged_line.append(merged_value)
                score_increase += merged_value
                skip = True
            else:
                merged_line.append(new_line[i])
        merged_line += [0] * (self.size - len(merged_line))
        return np.array(merged_line), score_increase

    def _move_board(self, direction):
        board = self.board.copy()
        total_reward = 0
        rotated_board = np.rot90(board, k=direction)
        new_board = np.zeros_like(rotated_board)
        for i in range(self.size):
            line = rotated_board[i, :]
            new_line, reward = self._move_line(line)
            new_board[i, :] = new_line
            total_reward += reward
        final_board = np.rot90(new_board, k=-direction)
        return final_board, total_reward

    def step(self, action):
        if self.done:
            return self.board.copy(), 0, True, {}
        new_board, reward = self._move_board(action)
        if not np.array_equal(new_board, self.board):
            self.board = new_board
            self._add_tile()
            self.score += reward
            if self.is_done():
                self.done = True
        else:
            reward = -10
        return self.board.copy(), reward, self.done, {}
    
    def _move(self, board, action):
        rotated_board = np.rot90(board, k=action)
        new_board = np.zeros_like(rotated_board)
        for i in range(self.size):
            line = rotated_board[i, :]
            new_line, reward = self._move_line(line)
            new_board[i, :] = new_line
        final_board = np.rot90(new_board, k=-action)
        return final_board
    
    def get_valid_actions(self):
        valid_actions = []
        for action in range(4):
            temp_board = self.board.copy()
             # Apply the move logic manually
            rotated_board = np.rot90(temp_board, k=action)
            new_board = np.zeros_like(rotated_board)
            for i in range(self.size):
                line = rotated_board[i, :]
                new_line, _ = self._move_line(line)
                new_board[i, :] = new_line
            final_board = np.rot90(new_board, k=-action)
            
            if not np.array_equal(final_board, temp_board):
                valid_actions.append(action)
        
        return valid_actions

    def is_done(self):
        return len(self.get_valid_actions()) == 0

# --- One-Hot Board Encoding ---
def encode_board(board):
    channels = 16 
    out = np.zeros((channels, 4, 4), dtype=np.float32)
    for r in range(4):
        for c in range(4):
            value = board[r][c]
            if value > 0:
                index = int(np.log2(value)) - 1
                if 0 <= index < channels:
                    out[index, r, c] = 1.0
    return out

# --- CNN-Based DQN ---
class CNN_DQN(nn.Module):
    def __init__(self, input_channels=16, output_dim=4):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(input_channels, 128, kernel_size=2, padding='same'),
            nn.ReLU(),
            nn.Conv2d(128, 256, kernel_size=2, padding='same'),
            nn.ReLU(),
        )
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256 * 4 * 4, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, output_dim)
        )

    def forward(self, x):
        x = self.conv(x)
        x = self.fc(x)
        return x

# --- OPTIMIZATION 1: Prioritized Experience Replay (PER) Buffer ---
class PrioritizedReplayBuffer:
    def __init__(self, capacity, alpha=0.6):
        self.capacity = capacity
        self.alpha = alpha
        self.buffer = []
        self.priorities = np.zeros(capacity, dtype=np.float32)
        self.position = 0

    def store(self, state, action, reward, next_state, done):
        max_prio = np.max(self.priorities) if self.buffer else 1.0
        
        experience = (state, action, reward, next_state, done)

        if len(self.buffer) < self.capacity:
            self.buffer.append(experience)
        else:
            self.buffer[self.position] = experience
            
        self.priorities[self.position] = max_prio
        self.position = (self.position + 1) % self.capacity

    def sample(self, batch_size, beta=0.4):
        if len(self.buffer) == self.capacity:
            prios = self.priorities
        else:
            prios = self.priorities[:self.position]
        
        probs = prios ** self.alpha
        probs /= probs.sum()

        indices = np.random.choice(len(self.buffer), batch_size, p=probs)
        samples = [self.buffer[i] for i in indices]

        total = len(self.buffer)
        weights = (total * probs[indices]) ** (-beta)
        weights /= weights.max()
        
        return samples, indices, np.array(weights, dtype=np.float32)

    def update_priorities(self, batch_indices, batch_priorities):
        for idx, prio in zip(batch_indices, batch_priorities):
            self.priorities[idx] = prio + 1e-5 # Add epsilon to avoid zero priority

    def __len__(self):
        return len(self.buffer)


# --- Agent with Double DQN ---
class Agent:
    def __init__(self, learning_rate=2.5e-5, gamma=0.99, max_memory=20000,
                 batch_size=128, epsilon_start=1.0, epsilon_min=0.01, epsilon_decay=0.9995, per_alpha=0.6, per_beta_start=0.4, per_beta_frames=200000):
        self.q_net = CNN_DQN()
        self.target_net = CNN_DQN()
        self.target_net.load_state_dict(self.q_net.state_dict())
        self.target_net.eval()
        self.optimizer = optim.Adam(self.q_net.parameters(), lr=learning_rate)
        self.gamma = gamma
        self.memory = PrioritizedReplayBuffer(max_memory, per_alpha)
        self.batch_size = batch_size
        self.epsilon = epsilon_start
        self.epsilon_min = epsilon_min
        self.epsilon_decay = epsilon_decay

        # Beta for PER importance sampling, annealed from start to 1 over training frames
        self.per_beta = per_beta_start
        self.per_beta_start = per_beta_start
        self.per_beta_frames = per_beta_frames
        self.frame_idx = 0

        # Learning rate scheduler
        self.scheduler = optim.lr_scheduler.StepLR(self.optimizer, step_size=10000, gamma=0.9)
        self.training_steps = 0

        # Training monitoring
        self.training_stats = {
            'losses': [],
            'td_errors': [],
            'q_values_mean': [],
            'targets_mean': [],
            'rewards_mean': [],
            'gradient_norms': []
        }

    def get_beta(self):
        return min(1.0, self.per_beta_start + self.frame_idx * (1.0 - self.per_beta_start) / self.per_beta_frames)

    def act(self, state_board, valid_actions):
        if not valid_actions:
             return 0
        self.frame_idx += 1
        if random.random() < self.epsilon:
            return random.choice(valid_actions)
        with torch.no_grad():
            state_tensor = torch.tensor(encode_board(state_board)).unsqueeze(0)
            q_values = self.q_net(state_tensor).squeeze().numpy()
        masked_q = np.full(q_values.shape, -np.inf)
        for a in valid_actions:
            masked_q[a] = q_values[a]
        return int(np.argmax(masked_q))

    def store(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def train_step(self):
        if len(self.memory) < self.batch_size:
            return
        
        beta = self.get_beta()
        batch, indices, weights = self.memory.sample(self.batch_size, beta)
        states, actions, rewards, next_states, dones = zip(*batch)
        states_enc = torch.tensor(np.array([encode_board(s) for s in states]), dtype=torch.float32)
        next_states_enc = torch.tensor(np.array([encode_board(s) for s in next_states]), dtype=torch.float32)
        actions = torch.tensor(actions, dtype=torch.int64)
        rewards = torch.tensor(rewards, dtype=torch.float32)
        dones = torch.tensor(dones, dtype=torch.float32)
        weights = torch.tensor(weights, dtype=torch.float32)

        # Calculate Q-values for the current state
        q_values = self.q_net(states_enc).gather(1, actions.unsqueeze(1)).squeeze(1)

        # Calculate Q-values for the next state
        with torch.no_grad():
            next_q_values_online = self.q_net(next_states_enc)
            # Get the best action for the next state
            best_next_actions = torch.argmax(next_q_values_online, dim=1)
            # Get the Q-values for the best action for the next state
            next_q_values_target = self.target_net(next_states_enc)
            next_q_target = next_q_values_target.gather(1, best_next_actions.unsqueeze(1)).squeeze(1)
            targets = rewards + self.gamma * next_q_target * (1 - dones)

        # Calculate the TD errors (target - q_values)
        td_errors = torch.abs(targets - q_values)

        # Calculate the loss
        loss = F.mse_loss(q_values, targets, reduction='none')
        loss = (loss * weights).mean()

        # Update the Q-network
        self.optimizer.zero_grad()
        loss.backward()
        gradient_norms = torch.nn.utils.clip_grad_norm_(self.q_net.parameters(), max_norm=1.0)
        self.optimizer.step()

        # Update the learning rate
        self.scheduler.step()
        self.training_steps += 1

        # Store the training stats
        self.store_training_stats(loss, td_errors, q_values, targets, rewards, gradient_norms)

        # Update the PER priorities
        priorities = td_errors.detach().clamp(max=5.0).cpu().numpy()
        self.memory.update_priorities(indices, priorities)

    
    # store the training stats
    def store_training_stats(self, loss, td_errors, q_values, targets, rewards, gradient_norms):
        self.training_stats['losses'].append(loss.item())
        self.training_stats['td_errors'].append(td_errors.mean().item())
        self.training_stats['q_values_mean'].append(q_values.mean().item())  # Take mean first
        self.training_stats['targets_mean'].append(targets.mean().item())    # Take mean first
        self.training_stats['rewards_mean'].append(rewards.mean().item())    # Take mean first
        self.training_stats['gradient_norms'].append(gradient_norms.item())
    
    def get_training_summary(self):
        """Get summary statistics of recent training"""
        if not self.training_stats['losses']:
            return None
            
        recent_losses = self.training_stats['losses'][-100:]  # Last 100 training steps
        recent_td_errors = self.training_stats['td_errors'][-100:]
        recent_q_values = self.training_stats['q_values_mean'][-100:]
        recent_targets = self.training_stats['targets_mean'][-100:]
        recent_rewards = self.training_stats['rewards_mean'][-100:]
        recent_gradient_norms = self.training_stats['gradient_norms'][-100:]
        
        return {
            'avg_loss': np.mean(recent_losses),
            'avg_td_error': np.mean(recent_td_errors),
            'loss_trend': np.mean(recent_losses[-20:]) - np.mean(recent_losses[:20]),  # Positive = increasing
            'td_error_trend': np.mean(recent_td_errors[-20:]) - np.mean(recent_td_errors[:20]),
            'total_training_steps': len(self.training_stats['losses'])
        }
    
    def update_target(self):
        self.target_net.load_state_dict(self.q_net.state_dict())

    def decay_epsilon(self):
        self.epsilon = max(self.epsilon_min, self.epsilon * self.epsilon_decay)


# --- Reward Shaping ---
def get_shaped_reward(reward_from_env, board, prev_board, done):
    if reward_from_env == -10:  # Invalid move
        return -10.0
    
    # Base reward from merges
    log_reward = 0
    if reward_from_env > 0:
        log_reward = np.log2(reward_from_env)
    
    # Bonus for reaching higher tiles
    max_tile = np.max(board)
    tile_bonus = np.log2(max_tile) * 0.5
    
    # Penalty for game over
    game_over_penalty = -50.0 if done else 0.0
    
    # Bonus for keeping high tiles in corners
    corner_bonus = 0
    if max_tile >= 128:
        corners = [board[0,0], board[0,3], board[3,0], board[3,3]]
        if max_tile in corners:
            corner_bonus = 2.0
    
    # Bonus for monotonicity (tiles decreasing from corner)
    monotonicity_bonus = calculate_monotonicity(board) * 0.1
    
    # Penalty for too many small tiles
    small_tile_penalty = -np.sum(board == 2) * 0.01
    
    return float(log_reward + tile_bonus + corner_bonus + monotonicity_bonus + small_tile_penalty + game_over_penalty)

def calculate_monotonicity(board):
    """Calculate how well tiles decrease from corner"""
    score = 0
    # Check horizontal monotonicity
    for i in range(4):
        for j in range(3):
            if board[i,j] >= board[i,j+1] and board[i,j] != 0:
                score += 1
    # Check vertical monotonicity  
    for i in range(3):
        for j in range(4):
            if board[i,j] >= board[i+1,j] and board[i,j] != 0:
                score += 1
    return score


### Training Loop

In [35]:
def training_to_reach_target(agent, env, target_tile, max_episodes=2000):
    # Training loop with better monitoring
    target_update_freq = 5  # More frequent updates
    best_tile = 0
    consecutive_no_improvement = 0

    for episode in range(1, max_episodes + 1):
        state = env.reset()
        done = False
        total_reward = 0
        steps = 0
        episode_training_steps = 0
        
        while not done:
            valid_actions = env.get_valid_actions()
            if not valid_actions:
                done = True
                continue
                
            action = agent.act(state, valid_actions)
            next_state, reward_from_env, done, _ = env.step(action)
            
            # Use shaped reward
            shaped_reward = get_shaped_reward(reward_from_env, next_state, state, done)
            agent.memory.store(state, action, shaped_reward, next_state, done)
            
            total_reward += shaped_reward
            state = next_state
            
            # Train more frequently
            if len(agent.memory) > agent.batch_size:
                agent.train_step()
                episode_training_steps += 1
            
            steps += 1
        
        agent.decay_epsilon()
        
        if episode % target_update_freq == 0:
            agent.update_target()
    
        
        max_tile = np.max(env.board)
        if max_tile > best_tile:
            best_tile = max_tile
            consecutive_no_improvement = 0
            print(f"*** New best tile: {best_tile} at episode {episode}! ***")
            torch.save(agent.q_net.state_dict(), f"cnn_dqn_best_{best_tile}.pth")
            if best_tile >= target_tile:
                print(f"Successfully reached {target_tile}!")
                return best_tile
        else:
            consecutive_no_improvement += 1
        
        # Early stopping if no improvement
        if consecutive_no_improvement > 500:
            print(f"No improvement for 500 episodes, stopping at tile {best_tile}")
            break
        
        if episode % 100 == 0:
            print(f"Ep {episode} | Steps: {steps} | Max Tile: {max_tile} | Reward: {int(total_reward)} | Epsilon: {agent.epsilon:.4f}")
    return best_tile

# --- Optimized Training Loop ---
env = Game2048Env()
agent = Agent(
    learning_rate=1e-4,  # Higher learning rate
    gamma=0.99,
    max_memory=50000,  # Larger buffer
    batch_size=256,  # Larger batches
    epsilon_start=1.0,
    epsilon_min=0.01,
    epsilon_decay=0.9999,  # Slower decay
    per_alpha=0.7,  # Higher priority
    per_beta_start=0.5
)

def curriculum_training():
    target_tiles = [64, 128, 256, 512, 1024, 2048]
    best_tile = 0
    
    # Create agent and environment
    env = Game2048Env()
    agent = Agent(
        learning_rate=1e-4,
        gamma=0.99,
        max_memory=50000,
        batch_size=256,
        epsilon_start=1.0,
        epsilon_min=0.01,
        epsilon_decay=0.9999,
        per_alpha=0.7,
        per_beta_start=0.5
    )
    
    for target in target_tiles:
        print(f"Training to reach {target} tile...")
        current_best = training_to_reach_target(agent, env, target)
        best_tile = max(best_tile, current_best)
        
        if best_tile >= target:
            print(f"Successfully reached {target}!")
        else:
            print(f"Could only reach {best_tile}, stopping curriculum")
            break
        
        print("=" * 50)
    
    print(f"Final best tile: {best_tile}")
    return agent, best_tile

# Run the training
agent, final_best_tile = curriculum_training()

Training to reach 64 tile...
*** New best tile: 16 at episode 1! ***
*** New best tile: 128 at episode 2! ***
Successfully reached 64!
Successfully reached 64!
Training to reach 128 tile...
*** New best tile: 128 at episode 1! ***
Successfully reached 128!
Successfully reached 128!
Training to reach 256 tile...
*** New best tile: 256 at episode 1! ***
Successfully reached 256!
Successfully reached 256!
Training to reach 512 tile...
*** New best tile: 32 at episode 1! ***
*** New best tile: 256 at episode 2! ***
Ep 100 | Steps: 153 | Max Tile: 128 | Reward: 878 | Epsilon: 0.9897


KeyboardInterrupt: 

#### 2048 PyGame Visualizer for DQN Agent


In [7]:
!pip3 install pygame


[notice] A new release of pip is available: 24.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [26]:
import pygame
import numpy as np
import torch

# Initialize Pygame
pygame.init()

# Constants
SIZE = 4
TILE_SIZE = 100
MARGIN = 10
WIDTH = SIZE * TILE_SIZE + (SIZE + 1) * MARGIN
HEIGHT = WIDTH
FONT = pygame.font.SysFont("arial", 32)

# Colors
BG_COLOR = (187, 173, 160)
TILE_COLORS = {
    0: (205, 193, 180),
    2: (238, 228, 218),
    4: (237, 224, 200),
    8: (242, 177, 121),
    16: (245, 149, 99),
    32: (246, 124, 95),
    64: (246, 94, 59),
    128: (237, 207, 114),
    256: (237, 204, 97),
    512: (237, 200, 80),
    1024: (237, 197, 63),
    2048: (237, 194, 46),
}
TEXT_COLOR_DARK = (119, 110, 101)
TEXT_COLOR_LIGHT = (249, 246, 242)

# Create screen
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("2048 Visualizer")

def draw_board(board):
    screen.fill(BG_COLOR)
    for row in range(SIZE):
        for col in range(SIZE):
            value = board[row][col]
            color = TILE_COLORS.get(value, (60, 58, 50))
            rect_x = MARGIN + col * (TILE_SIZE + MARGIN)
            rect_y = MARGIN + row * (TILE_SIZE + MARGIN)
            pygame.draw.rect(screen, color, (rect_x, rect_y, TILE_SIZE, TILE_SIZE))
            if value != 0:
                text_color = TEXT_COLOR_DARK if value <= 4 else TEXT_COLOR_LIGHT
                text = FONT.render(str(value), True, text_color)
                text_rect = text.get_rect(center=(rect_x + TILE_SIZE / 2, rect_y + TILE_SIZE / 2))
                screen.blit(text, text_rect)
    pygame.display.update()

# --- Agent Playing Loop ---
env = Game2048Env()
state = env.reset()
done = False
running = True

while running:
    # Handle quit event
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    if not done:
        valid_actions = env.get_valid_actions()

        if not valid_actions:
            done = True
            continue

        # Get Q-values from model
        with torch.no_grad():
            state_tensor = torch.tensor(encode_board(state)).unsqueeze(0)  # shape: (1, 16)
            q_values = agent.q_net(state_tensor).squeeze().numpy()

        # Mask invalid actions
        masked_q = np.full_like(q_values, -np.inf)
        for a in valid_actions:
            masked_q[a] = q_values[a]

        action = int(np.argmax(masked_q))

        # Step and draw
        state, reward, done, _ = env.step(action)
        draw_board(env.board)
        pygame.time.wait(200)
    else:
        # Show game over
        font = pygame.font.SysFont('arial', 40, bold=True)
        text = font.render(f"Game Over!", True, (255, 0, 0))
        text_rect = text.get_rect(center=(WIDTH / 2, HEIGHT / 2))
        screen.blit(text, text_rect)
        pygame.display.update()
        pygame.time.wait(2000)
        running = False

pygame.quit()

# Log final results
print("Final score:", np.sum(env.board))
print("Max tile:", np.max(env.board))


Final score: 326
Max tile: 128


#### Expectimax Algorithm

In [68]:
import numpy as np
import random
from collections import deque
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import numpy as np
import random

class Game2048Env:
    def __init__(self):
        self.size = 4
        self.reset()

    def reset(self):
        """Reset the game board and add two initial tiles"""
        self.board = np.zeros((self.size, self.size), dtype=int)
        self._add_random_tile()
        self._add_random_tile()
        self.score = 0
        self.done = False
        return self.board.copy()

    def _add_random_tile(self):
        """Add a random tile (2 or 4) to an empty position"""
        empty_positions = list(zip(*np.where(self.board == 0)))
        if not empty_positions:
            return
        
        row, col = random.choice(empty_positions)
        # 90% chance of 2, 10% chance of 4
        self.board[row, col] = 2 if random.random() < 0.9 else 4

    def _merge_line(self, line):
        """
        Merge a single line (row or column) to the left
        Returns: (merged_line, score_gained)
        """
        # Remove zeros
        non_zero = line[line != 0]
        merged = []
        score = 0
        i = 0
        
        while i < len(non_zero):
            if i + 1 < len(non_zero) and non_zero[i] == non_zero[i + 1]:
                # Merge identical adjacent tiles
                merged_value = non_zero[i] * 2
                merged.append(merged_value)
                score += merged_value
                i += 2  # Skip the next tile since it was merged
            else:
                merged.append(non_zero[i])
                i += 1
        
        # Pad with zeros to maintain original length
        merged.extend([0] * (len(line) - len(merged)))
        return np.array(merged), score

    def _move_left(self, board):
        """Move all tiles left"""
        new_board = np.zeros_like(board)
        total_score = 0
        
        for row in range(self.size):
            new_board[row, :], score = self._merge_line(board[row, :])
            total_score += score
            
        return new_board, total_score

    def _move_right(self, board):
        """Move all tiles right"""
        new_board = np.zeros_like(board)
        total_score = 0
        
        for row in range(self.size):
            # Reverse, merge left, then reverse back
            reversed_row = board[row, ::-1]
            merged_row, score = self._merge_line(reversed_row)
            new_board[row, :] = merged_row[::-1]
            total_score += score
            
        return new_board, total_score

    def _move_up(self, board):
        """Move all tiles up"""
        new_board = np.zeros_like(board)
        total_score = 0
        
        for col in range(self.size):
            new_board[:, col], score = self._merge_line(board[:, col])
            total_score += score
            
        return new_board, total_score

    def _move_down(self, board):
        """Move all tiles down"""
        new_board = np.zeros_like(board)
        total_score = 0
        
        for col in range(self.size):
            # Reverse, merge up, then reverse back
            reversed_col = board[::-1, col]
            merged_col, score = self._merge_line(reversed_col)
            new_board[:, col] = merged_col[::-1]
            total_score += score
            
        return new_board, total_score

    def _execute_move(self, board, action):
        """
        Execute a move on the board
        Actions: 0=UP, 1=DOWN, 2=LEFT, 3=RIGHT
        Returns: (new_board, score_gained)
        """
        if action == 0:    # UP
            return self._move_up(board)
        elif action == 1:  # DOWN
            return self._move_down(board)
        elif action == 2:  # LEFT
            return self._move_left(board)
        elif action == 3:  # RIGHT
            return self._move_right(board)
        else:
            raise ValueError(f"Invalid action: {action}. Must be 0-3.")

    def step(self, action):
        """
        Execute one game step
        Returns: (new_state, reward, done, info)
        """
        if self.done:
            return self.board.copy(), 0, True, {}

        # Try to execute the move
        new_board, reward = self._execute_move(self.board, action)
        
        # Check if the move actually changed the board
        if np.array_equal(new_board, self.board):
            # Invalid move - board didn't change
            return self.board.copy(), -10, self.done, {}
        
        # Valid move - update board and add new tile
        self.board = new_board
        self._add_random_tile()
        self.score += reward
        
        # Check if game is over
        if self.is_game_over():
            self.done = True
            
        return self.board.copy(), reward, self.done, {}

    def get_valid_actions(self):
        """Return list of valid actions (0=UP, 1=DOWN, 2=LEFT, 3=RIGHT)"""
        valid_actions = []
        
        for action in range(4):
            new_board, _ = self._execute_move(self.board, action)
            if not np.array_equal(new_board, self.board):
                valid_actions.append(action)
                
        return valid_actions

    def is_game_over(self):
        """Check if the game is over (no valid moves)"""
        return len(self.get_valid_actions()) == 0

    def get_max_tile(self):
        """Get the highest tile value on the board"""
        return np.max(self.board)

    def print_board(self):
        """Print the current board state"""
        print("Current board:")
        print(self.board)
        print(f"Score: {self.score}")
        print(f"Max tile: {self.get_max_tile()}")
        print(f"Valid actions: {self.get_valid_actions()}")
        print()


In [69]:
# Test the fixed implementation
env = Game2048Env()
env.board = np.array([
    [2, 2, 4, 8],
    [16, 32, 64, 128], 
    [256, 512, 1024, 2048],
    [16, 8, 2, 2]
])

print("Original board:")
print(env.board)
print()

# Test all directions
for action, name in [(0, "UP"), (1, "DOWN"), (2, "LEFT"), (3, "RIGHT")]:
    new_board, score = env._execute_move(env.board, action)
    print(f"{name} move:")
    print(new_board)
    print(f"Score gained: {score}")
    print(f"Board changed: {not np.array_equal(new_board, env.board)}")
    print()

Original board:
[[   2    2    4    8]
 [  16   32   64  128]
 [ 256  512 1024 2048]
 [  16    8    2    2]]

UP move:
[[   2    2    4    8]
 [  16   32   64  128]
 [ 256  512 1024 2048]
 [  16    8    2    2]]
Score gained: 0
Board changed: False

DOWN move:
[[   2    2    4    8]
 [  16   32   64  128]
 [ 256  512 1024 2048]
 [  16    8    2    2]]
Score gained: 0
Board changed: False

LEFT move:
[[   4    4    8    0]
 [  16   32   64  128]
 [ 256  512 1024 2048]
 [  16    8    4    0]]
Score gained: 8
Board changed: True

RIGHT move:
[[   0    4    4    8]
 [  16   32   64  128]
 [ 256  512 1024 2048]
 [   0   16    8    4]]
Score gained: 8
Board changed: True



In [71]:
class ExpectimaxAgent:
    def __init__(self, depth=4):
        self.depth = depth
        # Create a temporary environment for move simulation
        self.temp_env = Game2048Env()

    def get_action(self, board):
        """Get the best action using expectimax algorithm"""
        valid_actions = self.get_valid_actions(board)
        if not valid_actions:
            return 0  # No valid actions, return default
        
        _, best_move = self.expectimax(board, self.depth, True)
        # Ensure the best move is valid, otherwise pick first valid action
        return best_move if best_move in valid_actions else valid_actions[0]

    def get_valid_actions(self, board):
        """Get valid actions for a given board state"""
        valid_actions = []
        for action in range(4):
            new_board, _ = self.temp_env._execute_move(board, action)
            if not np.array_equal(new_board, board):
                valid_actions.append(action)
        return valid_actions

    def expectimax(self, board, depth, is_max):
        """
        Expectimax algorithm implementation
        is_max: True for player turn (maximizing), False for random tile placement (expectation)
        """
        if depth == 0 or self.is_terminal(board):
            return self.evaluate(board), None

        if is_max:
            # Player's turn - maximize
            best_value = float('-inf')
            best_move = None
            
            for action in range(4):  # 0=UP, 1=DOWN, 2=LEFT, 3=RIGHT
                new_board, _ = self.temp_env._execute_move(board, action)
                if not np.array_equal(new_board, board):  # Valid move
                    value, _ = self.expectimax(new_board, depth - 1, False)
                    if value > best_value:
                        best_value = value
                        best_move = action
                        
            return best_value if best_move is not None else float('-inf'), best_move
        else:
            # Random tile placement - expectation
            empty_positions = list(zip(*np.where(board == 0)))
            if not empty_positions:
                return self.evaluate(board), None

            expected_value = 0
            for (row, col) in empty_positions:
                for tile_value, probability in [(2, 0.9), (4, 0.1)]:
                    new_board = board.copy()
                    new_board[row, col] = tile_value
                    value, _ = self.expectimax(new_board, depth - 1, True)
                    expected_value += probability * value / len(empty_positions)
                    
            return expected_value, None

    def is_terminal(self, board):
        """Check if the board state is terminal (game over)"""
        # Game is terminal if no empty spaces and no valid moves
        if np.any(board == 0):
            return False  # Still has empty spaces
        return len(self.get_valid_actions(board)) == 0

    def evaluate(self, board):
        """
        Evaluation function for board states
        Consider multiple factors for better play
        """
        if self.is_terminal(board):
            return -1000  # Game over is bad
            
        # Basic evaluation components
        max_tile = np.max(board)
        empty_tiles = np.count_nonzero(board == 0)
        
        # Monotonicity bonus (higher tiles should be in corners/edges)
        monotonicity_score = self.calculate_monotonicity(board)
        
        # Smoothness bonus (adjacent tiles should have similar values)
        smoothness_score = self.calculate_smoothness(board)
        
        # Weighted combination
        score = (
            max_tile * 1.0 +           # Favor higher tiles
            empty_tiles * 2.7 +        # Favor more empty spaces
            monotonicity_score * 1.0 +  # Favor monotonic arrangements
            smoothness_score * 0.1      # Favor smooth transitions
        )
        
        return score

    def calculate_monotonicity(self, board):
        """Calculate monotonicity score - prefer tiles arranged in order"""
        totals = [0, 0, 0, 0]  # up, down, left, right
        
        for x in range(4):
            current = 0
            next_pos = 1
            while next_pos < 4:
                while next_pos < 4 and board[x][next_pos] == 0:
                    next_pos += 1
                if next_pos >= 4:
                    next_pos -= 1
                
                if board[x][current] != 0 and board[x][next_pos] != 0:
                    current_value = np.log2(board[x][current])
                    next_value = np.log2(board[x][next_pos])
                    
                    if current_value > next_value:
                        totals[0] += next_value - current_value
                    elif next_value > current_value:
                        totals[1] += current_value - next_value
                        
                current = next_pos
                next_pos += 1
        
        for y in range(4):
            current = 0
            next_pos = 1
            while next_pos < 4:
                while next_pos < 4 and board[next_pos][y] == 0:
                    next_pos += 1
                if next_pos >= 4:
                    next_pos -= 1
                    
                if board[current][y] != 0 and board[next_pos][y] != 0:
                    current_value = np.log2(board[current][y])
                    next_value = np.log2(board[next_pos][y])
                    
                    if current_value > next_value:
                        totals[2] += next_value - current_value
                    elif next_value > current_value:
                        totals[3] += current_value - next_value
                        
                current = next_pos
                next_pos += 1
                
        return max(totals[0], totals[1]) + max(totals[2], totals[3])

    def calculate_smoothness(self, board):
        """Calculate smoothness score - prefer similar adjacent values"""
        smoothness = 0
        
        for x in range(4):
            for y in range(4):
                if board[x][y] != 0:
                    value = np.log2(board[x][y])
                    # Check right neighbor
                    if y + 1 < 4 and board[x][y + 1] != 0:
                        target_value = np.log2(board[x][y + 1])
                        smoothness -= abs(value - target_value)
                    # Check down neighbor  
                    if x + 1 < 4 and board[x + 1][y] != 0:
                        target_value = np.log2(board[x + 1][y])
                        smoothness -= abs(value - target_value)
                        
        return smoothness

In [72]:
# Test the updated ExpectimaxAgent
env = Game2048Env()
agent = ExpectimaxAgent(depth=3)

env.board = np.array([
    [2, 2, 4, 8],
    [16, 32, 64, 128], 
    [256, 512, 1024, 2048],
    [16, 8, 2, 2]
])

print("Board:")
print(env.board)
print(f"Valid actions: {env.get_valid_actions()}")
print(f"Agent's choice: {agent.get_action(env.board)}")

# Actions: 0=UP, 1=DOWN, 2=LEFT, 3=RIGHT
action_names = ["UP", "DOWN", "LEFT", "RIGHT"]
action = agent.get_action(env.board)
print(f"Agent chooses: {action_names[action]}")

Board:
[[   2    2    4    8]
 [  16   32   64  128]
 [ 256  512 1024 2048]
 [  16    8    2    2]]
Valid actions: [2, 3]
Agent's choice: 3
Agent chooses: RIGHT


#### PyGame Visualizer for Expectimax Algorithm

In [73]:
import pygame
import numpy as np
import time
import random

# Initialize Pygame
pygame.init()

# Constants
SIZE = 4
TILE_SIZE = 100
MARGIN = 10
WIDTH = SIZE * TILE_SIZE + (SIZE + 1) * MARGIN
HEIGHT = WIDTH
FONT = pygame.font.SysFont("arial", 32)

# Colors
BG_COLOR = (187, 173, 160)
TILE_COLORS = {
    0: (205, 193, 180),
    2: (238, 228, 218),
    4: (237, 224, 200),
    8: (242, 177, 121),
    16: (245, 149, 99),
    32: (246, 124, 95),
    64: (246, 94, 59),
    128: (237, 207, 114),
    256: (237, 204, 97),
    512: (237, 200, 80),
    1024: (237, 197, 63),
    2048: (237, 194, 46),
}
TEXT_COLOR_DARK = (119, 110, 101)
TEXT_COLOR_LIGHT = (249, 246, 242)

# Setup display
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("2048 Expectimax")

def draw_board(board):
    screen.fill(BG_COLOR)
    for row in range(SIZE):
        for col in range(SIZE):
            value = board[row][col]
            color = TILE_COLORS.get(value, (60, 58, 50))
            rect_x = MARGIN + col * (TILE_SIZE + MARGIN)
            rect_y = MARGIN + row * (TILE_SIZE + MARGIN)
            pygame.draw.rect(screen, color, (rect_x, rect_y, TILE_SIZE, TILE_SIZE))
            if value != 0:
                text_color = TEXT_COLOR_DARK if value <= 4 else TEXT_COLOR_LIGHT
                text = FONT.render(str(value), True, text_color)
                text_rect = text.get_rect(center=(rect_x + TILE_SIZE / 2, rect_y + TILE_SIZE / 2))
                screen.blit(text, text_rect)
    pygame.display.update()

# --- Expectimax playing loop ---

env = Game2048Env()
agent = ExpectimaxAgent(depth=3)
state = env.reset()
done = False
running = True

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    if not done:
        draw_board(env.board)
        time.sleep(0.2)

        valid_actions = env.get_valid_actions()
        if not valid_actions:
            done = True
            continue

        action = agent.get_action(env.board)
        if action not in valid_actions:
            print("Expectimax chose invalid action, picking random.")
            action = random.choice(valid_actions)

        state, reward, done, _ = env.step(action)
    else:
        font = pygame.font.SysFont('arial', 40, bold=True)
        text = font.render(f"Game Over!", True, (255, 0, 0))
        text_rect = text.get_rect(center=(WIDTH / 2, HEIGHT / 2))
        screen.blit(text, text_rect)
        pygame.display.update()
        pygame.time.wait(2000)
        running = False

pygame.quit()
print("Final score:", np.sum(env.board))
print("Max tile:", np.max(env.board))


Final score: 1608
Max tile: 1024


In [80]:
class SuperExpectimaxAgent:
    def __init__(self, depth=5):
        self.depth = depth
        self.temp_env = Game2048Env()
        
        # Memoization for speed
        self.memo = {}
        self.memo_hits = 0
        self.memo_misses = 0

    def get_action(self, board):
        """Get best action using advanced expectimax"""
        # Clear memo periodically to prevent memory issues
        if len(self.memo) > 50000:
            self.memo.clear()
            
        valid_actions = self.get_valid_actions(board)
        if not valid_actions:
            return 0
        
        best_value = float('-inf')
        best_action = valid_actions[0]
        
        for action in valid_actions:
            new_board, _ = self.temp_env._execute_move(board, action)
            value, _ = self.expectimax(new_board, self.depth - 1, False)
            if value > best_value:
                best_value = value
                best_action = action
                
        return best_action

    def get_valid_actions(self, board):
        """Get valid actions"""
        valid_actions = []
        for action in range(4):
            new_board, _ = self.temp_env._execute_move(board, action)
            if not np.array_equal(new_board, board):
                valid_actions.append(action)
        return valid_actions

    def expectimax(self, board, depth, is_max):
        """Expectimax with memoization"""
        # Create hash for memoization
        board_hash = hash(board.tobytes())
        memo_key = (board_hash, depth, is_max)
        
        if memo_key in self.memo:
            self.memo_hits += 1
            return self.memo[memo_key]
        
        self.memo_misses += 1
        
        if depth == 0 or self.is_terminal(board):
            result = self.evaluate(board), None
            self.memo[memo_key] = result
            return result

        if is_max:
            best_value = float('-inf')
            best_move = None
            
            for action in self.get_valid_actions(board):
                new_board, _ = self.temp_env._execute_move(board, action)
                value, _ = self.expectimax(new_board, depth - 1, False)
                if value > best_value:
                    best_value = value
                    best_move = action
                    
            result = best_value if best_move is not None else float('-inf'), best_move
        else:
            empty_positions = list(zip(*np.where(board == 0)))
            if not empty_positions:
                result = self.evaluate(board), None
            else:
                expected_value = 0
                for (row, col) in empty_positions:
                    for tile_value, probability in [(2, 0.9), (4, 0.1)]:
                        new_board = board.copy()
                        new_board[row, col] = tile_value
                        value, _ = self.expectimax(new_board, depth - 1, True)
                        expected_value += probability * value / len(empty_positions)
                result = expected_value, None
                
        self.memo[memo_key] = result
        return result

    def is_terminal(self, board):
        """Check if game is over"""
        if np.any(board == 0):
            return False
        return len(self.get_valid_actions(board)) == 0

    def evaluate(self, board):
        """
        MUCH BETTER evaluation function for reaching 2048
        """
        if self.is_terminal(board):
            return -100000  # Severe penalty for game over
            
        # Core metrics
        max_tile = np.max(board)
        empty_tiles = np.count_nonzero(board == 0)
        
        # 1. CORNER STRATEGY - This is crucial!
        corner_score = self.calculate_corner_score(board)
        
        # 2. SNAKE PATTERN - Keep tiles in monotonic order
        snake_score = self.calculate_snake_score(board)
        
        # 3. EDGE WEIGHT - Prefer high tiles on edges
        edge_score = self.calculate_edge_score(board)
        
        # 4. CLUSTERING - Penalize scattered high tiles
        clustering_penalty = self.calculate_clustering_penalty(board)
        
        # 5. MERGE POTENTIAL - Reward potential merges
        merge_potential = self.calculate_merge_potential(board)
        
        # 6. SMOOTHNESS - Penalize large gaps between adjacent tiles
        smoothness_penalty = self.calculate_smoothness_penalty(board)
        
        # OPTIMIZED WEIGHTS for reaching 2048
        score = (
            max_tile * 1.0 +                    # Base tile value
            empty_tiles * 100.0 +               # Empty spaces are CRUCIAL
            corner_score * 1000.0 +             # Corner strategy is KEY
            snake_score * 200.0 +               # Snake pattern helps a lot
            edge_score * 50.0 +                 # Edge preference
            clustering_penalty * -500.0 +       # Avoid scattered tiles
            merge_potential * 80.0 +            # Reward merge opportunities
            smoothness_penalty * -30.0          # Smooth transitions
        )
        
        return score

    def calculate_corner_score(self, board):
        """Enhanced corner scoring"""
        max_val = np.max(board)
        
        # Find position of max tile
        max_pos = np.unravel_index(np.argmax(board), board.shape)
        
        # HUGE bonus if max tile is in corner
        if max_pos in [(0,0), (0,3), (3,0), (3,3)]:
            bonus = max_val * 10
            
            # Extra bonus for specific corner preferences
            if max_pos == (0,0):  # Top-left preferred
                bonus *= 1.5
            elif max_pos == (3,0):  # Bottom-left second choice
                bonus *= 1.3
                
            return bonus
        
        # Medium bonus if on edge
        row, col = max_pos
        if row == 0 or row == 3 or col == 0 or col == 3:
            return max_val * 3
            
        # Penalty if in middle
        return -max_val * 2

    def calculate_snake_score(self, board):
        """Calculate snake pattern score"""
        # Define snake patterns from each corner
        patterns = {
            'top_left': [
                (0,0), (0,1), (0,2), (0,3),
                (1,3), (1,2), (1,1), (1,0),
                (2,0), (2,1), (2,2), (2,3),
                (3,3), (3,2), (3,1), (3,0)
            ],
            'top_right': [
                (0,3), (0,2), (0,1), (0,0),
                (1,0), (1,1), (1,2), (1,3),
                (2,3), (2,2), (2,1), (2,0),
                (3,0), (3,1), (3,2), (3,3)
            ],
            'bottom_left': [
                (3,0), (3,1), (3,2), (3,3),
                (2,3), (2,2), (2,1), (2,0),
                (1,0), (1,1), (1,2), (1,3),
                (0,3), (0,2), (0,1), (0,0)
            ],
            'bottom_right': [
                (3,3), (3,2), (3,1), (3,0),
                (2,0), (2,1), (2,2), (2,3),
                (1,3), (1,2), (1,1), (1,0),
                (0,0), (0,1), (0,2), (0,3)
            ]
        }
        
        best_score = 0
        for pattern in patterns.values():
            score = 0
            for i in range(len(pattern) - 1):
                r1, c1 = pattern[i]
                r2, c2 = pattern[i + 1]
                
                if board[r1, c1] != 0 and board[r2, c2] != 0:
                    # Reward decreasing pattern
                    if board[r1, c1] >= board[r2, c2]:
                        score += min(board[r1, c1], board[r2, c2])
                    else:
                        score -= abs(board[r1, c1] - board[r2, c2])
                        
            best_score = max(best_score, score)
            
        return best_score

    def calculate_edge_score(self, board):
        """Reward high values on edges"""
        edge_score = 0
        
        # Top and bottom edges
        for col in range(4):
            edge_score += board[0, col] * 2  # Top edge
            edge_score += board[3, col] * 2  # Bottom edge
            
        # Left and right edges  
        for row in range(4):
            edge_score += board[row, 0] * 2  # Left edge
            edge_score += board[row, 3] * 2  # Right edge
            
        return edge_score

    def calculate_clustering_penalty(self, board):
        """Penalize high tiles that are isolated"""
        penalty = 0
        
        for row in range(4):
            for col in range(4):
                if board[row, col] >= 64:  # High tiles
                    # Check if surrounded by much smaller tiles
                    neighbors = []
                    for dr, dc in [(-1,0), (1,0), (0,-1), (0,1)]:
                        nr, nc = row + dr, col + dc
                        if 0 <= nr < 4 and 0 <= nc < 4:
                            neighbors.append(board[nr, nc])
                    
                    if neighbors:
                        avg_neighbor = sum(neighbors) / len(neighbors)
                        if avg_neighbor < board[row, col] / 4:  # Much smaller neighbors
                            penalty += board[row, col]
                            
        return penalty

    def calculate_merge_potential(self, board):
        """Calculate potential for merges"""
        potential = 0
        
        # Check horizontal merges
        for row in range(4):
            for col in range(3):
                if board[row, col] != 0 and board[row, col] == board[row, col + 1]:
                    potential += board[row, col] * 2
                    
        # Check vertical merges
        for row in range(3):
            for col in range(4):
                if board[row, col] != 0 and board[row, col] == board[row + 1, col]:
                    potential += board[row, col] * 2
                    
        return potential

    def calculate_smoothness_penalty(self, board):
        """Penalty for rough transitions"""
        penalty = 0
        
        for row in range(4):
            for col in range(3):
                if board[row, col] != 0 and board[row, col + 1] != 0:
                    diff = abs(np.log2(board[row, col]) - np.log2(board[row, col + 1]))
                    penalty += diff ** 2  # Quadratic penalty
                    
        for row in range(3):
            for col in range(4):
                if board[row, col] != 0 and board[row + 1, col] != 0:
                    diff = abs(np.log2(board[row, col]) - np.log2(board[row + 1, col]))
                    penalty += diff ** 2  # Quadratic penalty
                    
        return penalty

# Test the super agent
print("=== Testing Super Expectimax Agent ===")
super_agent = SuperExpectimaxAgent(depth=5)

# Quick test function
def quick_test_agent(agent, max_moves=2000):
    """Quick test without visualization"""
    env = Game2048Env()
    state = env.reset()
    moves = 0
    
    while not env.done and moves < max_moves:
        valid_actions = env.get_valid_actions()
        if not valid_actions:
            break
            
        action = agent.get_action(env.board)
        state, reward, done, _ = env.step(action)
        moves += 1
        
        # Print progress
        if moves % 100 == 0:
            print(f"Move {moves}: Max tile = {env.get_max_tile()}")
            
        # Stop if we reach 2048!
        if env.get_max_tile() >= 2048:
            print(f"🎉 SUCCESS! Reached 2048 in {moves} moves! 🎉")
            break
    
    return env.get_max_tile(), env.score, moves

# Run the test
max_tile, score, moves = quick_test_agent(super_agent)
print(f"Final results: Max tile = {max_tile}, Score = {score}, Moves = {moves}")

if hasattr(super_agent, 'memo_hits'):
    total_calls = super_agent.memo_hits + super_agent.memo_misses
    hit_rate = super_agent.memo_hits / total_calls if total_calls > 0 else 0
    print(f"Memoization hit rate: {hit_rate:.2%} ({super_agent.memo_hits}/{total_calls})")

=== Testing Super Expectimax Agent ===
Move 100: Max tile = 128
Move 200: Max tile = 256
Move 300: Max tile = 512
Move 400: Max tile = 512
Move 500: Max tile = 1024
Move 600: Max tile = 1024
Move 700: Max tile = 1024
Move 800: Max tile = 1024
Move 900: Max tile = 1024
Final results: Max tile = 1024, Score = 15612, Moves = 921
Memoization hit rate: 50.16% (1704027/3397286)


In [81]:
class UltimateExpectimaxAgent:
    def __init__(self, depth=6):
        self.depth = depth
        self.temp_env = Game2048Env()
        
        # Advanced memoization with better cache management
        self.memo = {}
        self.memo_hits = 0
        self.memo_misses = 0
        
        # Adaptive depth based on game state
        self.adaptive_depth = True
        
        # Endgame detection threshold
        self.endgame_threshold = 8  # When fewer than 8 empty spaces
        
        # Opening book for early game
        self.opening_moves = {
            # Start by going to corners
            hash(np.array([[2, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 2]]).tobytes()): 2,  # LEFT
            hash(np.array([[2, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [2, 0, 0, 0]]).tobytes()): 1,  # DOWN
        }

    def get_action(self, board):
        """Enhanced action selection with adaptive strategies"""
        # Clear memo periodically
        if len(self.memo) > 100000:
            self.memo.clear()
            
        # Check opening book first
        board_hash = hash(board.tobytes())
        if board_hash in self.opening_moves:
            return self.opening_moves[board_hash]
            
        valid_actions = self.get_valid_actions(board)
        if not valid_actions:
            return 0
        
        # Adaptive depth based on game state
        current_depth = self.get_adaptive_depth(board)
        
        best_value = float('-inf')
        best_action = valid_actions[0]
        
        # Try actions in smart order (corner-preserving moves first)
        ordered_actions = self.order_actions(board, valid_actions)
        
        for action in ordered_actions:
            new_board, _ = self.temp_env._execute_move(board, action)
            value, _ = self.expectimax(new_board, current_depth - 1, False)
            if value > best_value:
                best_value = value
                best_action = action
                
        return best_action

    def get_adaptive_depth(self, board):
        """Adjust search depth based on game state"""
        empty_tiles = np.count_nonzero(board == 0)
        max_tile = np.max(board)
        
        # Deeper search in critical situations
        if empty_tiles <= 4:  # Endgame
            return min(8, self.depth + 2)
        elif empty_tiles <= 8:  # Mid-late game
            return self.depth + 1
        elif max_tile >= 512:  # High value tiles present
            return self.depth
        else:  # Early game
            return max(4, self.depth - 1)

    def order_actions(self, board, valid_actions):
        """Order actions by priority (corner-preserving first)"""
        max_tile = np.max(board)
        max_pos = np.unravel_index(np.argmax(board), board.shape)
        
        # Priority based on max tile position
        if max_pos == (0, 0):  # Top-left corner
            priority = [2, 1, 0, 3]  # LEFT, DOWN, UP, RIGHT
        elif max_pos == (0, 3):  # Top-right corner
            priority = [3, 1, 0, 2]  # RIGHT, DOWN, UP, LEFT
        elif max_pos == (3, 0):  # Bottom-left corner
            priority = [2, 0, 1, 3]  # LEFT, UP, DOWN, RIGHT
        elif max_pos == (3, 3):  # Bottom-right corner
            priority = [3, 0, 1, 2]  # RIGHT, UP, DOWN, LEFT
        else:  # Max tile not in corner - try to move it there
            priority = [2, 1, 0, 3]  # Default to LEFT priority
            
        # Sort valid actions by priority
        return sorted(valid_actions, key=lambda x: priority.index(x) if x in priority else 999)

    def expectimax(self, board, depth, is_max):
        """Enhanced expectimax with alpha-beta style pruning"""
        board_hash = hash(board.tobytes())
        memo_key = (board_hash, depth, is_max)
        
        if memo_key in self.memo:
            self.memo_hits += 1
            return self.memo[memo_key]
        
        self.memo_misses += 1
        
        if depth == 0 or self.is_terminal(board):
            result = self.evaluate(board), None
            self.memo[memo_key] = result
            return result

        if is_max:
            best_value = float('-inf')
            best_move = None
            
            actions = self.order_actions(board, self.get_valid_actions(board))
            
            for action in actions:
                new_board, _ = self.temp_env._execute_move(board, action)
                value, _ = self.expectimax(new_board, depth - 1, False)
                if value > best_value:
                    best_value = value
                    best_move = action
                    
            result = best_value if best_move is not None else float('-inf'), best_move
        else:
            empty_positions = list(zip(*np.where(board == 0)))
            if not empty_positions:
                result = self.evaluate(board), None
            else:
                expected_value = 0
                # Sample fewer positions if too many empty tiles (performance optimization)
                if len(empty_positions) > 6:
                    empty_positions = random.sample(empty_positions, 6)
                    
                for (row, col) in empty_positions:
                    for tile_value, probability in [(2, 0.9), (4, 0.1)]:
                        new_board = board.copy()
                        new_board[row, col] = tile_value
                        value, _ = self.expectimax(new_board, depth - 1, True)
                        expected_value += probability * value / len(empty_positions)
                result = expected_value, None
                
        self.memo[memo_key] = result
        return result

    def get_valid_actions(self, board):
        """Get valid actions"""
        valid_actions = []
        for action in range(4):
            new_board, _ = self.temp_env._execute_move(board, action)
            if not np.array_equal(new_board, board):
                valid_actions.append(action)
        return valid_actions

    def is_terminal(self, board):
        """Enhanced terminal detection"""
        if np.any(board == 0):
            return False
        return len(self.get_valid_actions(board)) == 0

    def evaluate(self, board):
        """
        ULTIMATE evaluation function - heavily tuned for 2048
        """
        if self.is_terminal(board):
            return -1000000  # Massive penalty for game over
            
        empty_tiles = np.count_nonzero(board == 0)
        max_tile = np.max(board)
        
        # CRITICAL: If we're close to 2048, prioritize that above all else
        if max_tile >= 1024:
            return self.evaluate_endgame(board)
        
        # Core components
        corner_score = self.calculate_corner_score(board)
        snake_score = self.calculate_snake_score(board)
        edge_score = self.calculate_edge_score(board)
        clustering_penalty = self.calculate_clustering_penalty(board)
        merge_potential = self.calculate_merge_potential(board)
        smoothness_penalty = self.calculate_smoothness_penalty(board)
        
        # TUNED WEIGHTS - heavily favor corner strategy and empty spaces
        score = (
            max_tile * 1.0 +                    
            empty_tiles * 200.0 +               # INCREASED: Empty spaces crucial
            corner_score * 2000.0 +             # INCREASED: Corner strategy essential
            snake_score * 300.0 +               # INCREASED: Snake pattern
            edge_score * 75.0 +                 
            clustering_penalty * -800.0 +       # INCREASED: Avoid scattered tiles
            merge_potential * 150.0 +           # INCREASED: Merge opportunities
            smoothness_penalty * -50.0          
        )
        
        return score

    def evaluate_endgame(self, board):
        """Special evaluation for when max tile >= 1024"""
        empty_tiles = np.count_nonzero(board == 0)
        max_tile = np.max(board)
        
        # Find the 1024 tile position
        max_pos = np.unravel_index(np.argmax(board), board.shape)
        
        # MASSIVE bonus if 1024 is in corner
        if max_pos in [(0,0), (0,3), (3,0), (3,3)]:
            corner_bonus = max_tile * 50  # 50x multiplier!
        else:
            corner_bonus = -max_tile * 10  # Heavy penalty if not in corner
            
        # Check for 1024 merge potential (path to 2048!)
        merge_1024_bonus = 0
        for dr, dc in [(-1,0), (1,0), (0,-1), (0,1)]:
            nr, nc = max_pos[0] + dr, max_pos[1] + dc
            if 0 <= nr < 4 and 0 <= nc < 4:
                if board[nr, nc] == 512:  # Can merge to make 2048!
                    merge_1024_bonus = 100000  # HUGE bonus
                    break
        
        # Conservative play - preserve empty spaces at all costs
        empty_bonus = empty_tiles * 1000  # 5x normal weight
        
        # Penalize any risky moves heavily
        risk_penalty = 0
        if empty_tiles <= 3:
            risk_penalty = -50000  # Very risky state
        elif empty_tiles <= 5:
            risk_penalty = -10000  # Somewhat risky
            
        return corner_bonus + merge_1024_bonus + empty_bonus + risk_penalty

    # [Keep all the existing heuristic methods but with small improvements]
    def calculate_corner_score(self, board):
        """Enhanced corner scoring with stronger preferences"""
        max_val = np.max(board)
        max_pos = np.unravel_index(np.argmax(board), board.shape)
        
        if max_pos in [(0,0), (0,3), (3,0), (3,3)]:
            bonus = max_val * 15  # Increased from 10
            if max_pos == (0,0):  # Top-left preferred
                bonus *= 2.0  # Increased from 1.5
            elif max_pos == (3,0):  # Bottom-left second choice
                bonus *= 1.8  # Increased from 1.3
            return bonus
        
        row, col = max_pos
        if row == 0 or row == 3 or col == 0 or col == 3:
            return max_val * 5  # Increased from 3
            
        return -max_val * 5  # Increased penalty from 2

    def calculate_snake_score(self, board):
        """Keep existing implementation"""
        patterns = {
            'top_left': [
                (0,0), (0,1), (0,2), (0,3),
                (1,3), (1,2), (1,1), (1,0),
                (2,0), (2,1), (2,2), (2,3),
                (3,3), (3,2), (3,1), (3,0)
            ]
        }
        
        best_score = 0
        for pattern in patterns.values():
            score = 0
            for i in range(len(pattern) - 1):
                r1, c1 = pattern[i]
                r2, c2 = pattern[i + 1]
                
                if board[r1, c1] != 0 and board[r2, c2] != 0:
                    if board[r1, c1] >= board[r2, c2]:
                        score += min(board[r1, c1], board[r2, c2])
                    else:
                        score -= abs(board[r1, c1] - board[r2, c2])
                        
            best_score = max(best_score, score)
            
        return best_score

    def calculate_edge_score(self, board):
        """Keep existing implementation"""
        edge_score = 0
        for col in range(4):
            edge_score += board[0, col] * 2
            edge_score += board[3, col] * 2
        for row in range(4):
            edge_score += board[row, 0] * 2
            edge_score += board[row, 3] * 2
        return edge_score

    def calculate_clustering_penalty(self, board):
        """Keep existing implementation"""
        penalty = 0
        for row in range(4):
            for col in range(4):
                if board[row, col] >= 64:
                    neighbors = []
                    for dr, dc in [(-1,0), (1,0), (0,-1), (0,1)]:
                        nr, nc = row + dr, col + dc
                        if 0 <= nr < 4 and 0 <= nc < 4:
                            neighbors.append(board[nr, nc])
                    
                    if neighbors:
                        avg_neighbor = sum(neighbors) / len(neighbors)
                        if avg_neighbor < board[row, col] / 4:
                            penalty += board[row, col]
        return penalty

    def calculate_merge_potential(self, board):
        """Enhanced to prioritize high-value merges"""
        potential = 0
        
        for row in range(4):
            for col in range(3):
                if board[row, col] != 0 and board[row, col] == board[row, col + 1]:
                    merge_value = board[row, col] * 2
                    # Extra bonus for high-value merges
                    if merge_value >= 512:
                        potential += merge_value * 3
                    else:
                        potential += merge_value
                        
        for row in range(3):
            for col in range(4):
                if board[row, col] != 0 and board[row, col] == board[row + 1, col]:
                    merge_value = board[row, col] * 2
                    if merge_value >= 512:
                        potential += merge_value * 3
                    else:
                        potential += merge_value
                        
        return potential

    def calculate_smoothness_penalty(self, board):
        """Keep existing implementation"""
        penalty = 0
        for row in range(4):
            for col in range(3):
                if board[row, col] != 0 and board[row, col + 1] != 0:
                    diff = abs(np.log2(board[row, col]) - np.log2(board[row, col + 1]))
                    penalty += diff ** 2
                    
        for row in range(3):
            for col in range(4):
                if board[row, col] != 0 and board[row + 1, col] != 0:
                    diff = abs(np.log2(board[row, col]) - np.log2(board[row + 1, col]))
                    penalty += diff ** 2
        return penalty

# Test multiple runs to find best strategy
def test_multiple_runs(agent_class, num_runs=5):
    """Run multiple games and report statistics"""
    results = []
    
    for run in range(num_runs):
        print(f"\n=== Run {run + 1}/{num_runs} ===")
        agent = agent_class(depth=5)
        
        env = Game2048Env()
        state = env.reset()
        moves = 0
        
        while not env.done and moves < 2000:
            valid_actions = env.get_valid_actions()
            if not valid_actions:
                break
                
            action = agent.get_action(env.board)
            state, reward, done, _ = env.step(action)
            moves += 1
            
            if moves % 200 == 0:
                print(f"Move {moves}: Max tile = {env.get_max_tile()}")
                
            if env.get_max_tile() >= 2048:
                print(f"🎉 SUCCESS! Reached 2048 in {moves} moves! 🎉")
                break
        
        result = {
            'max_tile': env.get_max_tile(),
            'score': env.score,
            'moves': moves,
            'reached_2048': env.get_max_tile() >= 2048
        }
        results.append(result)
        print(f"Result: Max tile = {result['max_tile']}, Score = {result['score']}")
    
    # Print summary
    print(f"\n=== SUMMARY OF {num_runs} RUNS ===")
    max_tiles = [r['max_tile'] for r in results]
    scores = [r['score'] for r in results]
    success_rate = sum(1 for r in results if r['reached_2048']) / num_runs
    
    print(f"Success rate (2048): {success_rate:.1%}")
    print(f"Average max tile: {np.mean(max_tiles):.0f}")
    print(f"Best max tile: {max(max_tiles)}")
    print(f"Average score: {np.mean(scores):.0f}")
    print(f"Best score: {max(scores)}")
    
    return results

# Run the ultimate test
print("=== Testing Ultimate Expectimax Agent ===")
results = test_multiple_runs(UltimateExpectimaxAgent, num_runs=3)

=== Testing Ultimate Expectimax Agent ===

=== Run 1/3 ===
Move 200: Max tile = 256
Move 400: Max tile = 512
Move 600: Max tile = 1024
Move 800: Max tile = 1024
🎉 SUCCESS! Reached 2048 in 980 moves! 🎉
Result: Max tile = 2048, Score = 20436

=== Run 2/3 ===
Move 200: Max tile = 256
Move 400: Max tile = 512
Move 600: Max tile = 512
Move 800: Max tile = 1024
Result: Max tile = 1024, Score = 15148

=== Run 3/3 ===
Move 200: Max tile = 256
Move 400: Max tile = 512
Move 600: Max tile = 512
Result: Max tile = 512, Score = 11016

=== SUMMARY OF 3 RUNS ===
Success rate (2048): 33.3%
Average max tile: 1195
Best max tile: 2048
Average score: 15533
Best score: 20436


In [ ]:
# Pygame visualization for the fixed agent
import pygame
import time

def visualize_agent_gameplay(agent, delay=0.5):
    """Visualize agent playing with pygame"""
    pygame.init()
    
    # Constants
    SIZE = 4
    TILE_SIZE = 100
    MARGIN = 10
    WIDTH = SIZE * TILE_SIZE + (SIZE + 1) * MARGIN
    HEIGHT = WIDTH + 100  # Extra space for text
    FONT = pygame.font.SysFont("arial", 32)
    INFO_FONT = pygame.font.SysFont("arial", 24)

    # Colors
    BG_COLOR = (187, 173, 160)
    TILE_COLORS = {
        0: (205, 193, 180), 2: (238, 228, 218), 4: (237, 224, 200),
        8: (242, 177, 121), 16: (245, 149, 99), 32: (246, 124, 95),
        64: (246, 94, 59), 128: (237, 207, 114), 256: (237, 204, 97),
        512: (237, 200, 80), 1024: (237, 197, 63), 2048: (237, 194, 46),
        4096: (237, 194, 46)
    }
    TEXT_COLOR_DARK = (119, 110, 101)
    TEXT_COLOR_LIGHT = (249, 246, 242)

    screen = pygame.display.set_mode((WIDTH, HEIGHT))
    pygame.display.set_caption("2048 AI Agent")

    def draw_game_state(env, move_count):
        screen.fill(BG_COLOR)
        
        # Draw board
        for row in range(SIZE):
            for col in range(SIZE):
                value = env.board[row][col]
                color = TILE_COLORS.get(value, (60, 58, 50))
                rect_x = MARGIN + col * (TILE_SIZE + MARGIN)
                rect_y = MARGIN + row * (TILE_SIZE + MARGIN)
                pygame.draw.rect(screen, color, (rect_x, rect_y, TILE_SIZE, TILE_SIZE))
                
                if value != 0:
                    text_color = TEXT_COLOR_DARK if value <= 4 else TEXT_COLOR_LIGHT
                    text = FONT.render(str(value), True, text_color)
                    text_rect = text.get_rect(center=(rect_x + TILE_SIZE / 2, rect_y + TILE_SIZE / 2))
                    screen.blit(text, text_rect)
        
        # Draw info
        info_y = WIDTH + 10
        info_text = INFO_FONT.render(f"Move: {move_count} | Score: {env.score} | Max: {env.get_max_tile()}", True, (0, 0, 0))
        screen.blit(info_text, (10, info_y))
        
        pygame.display.update()

    # Run game
    env = Game2048Env()
    state = env.reset()
    move_count = 0
    running = True

    while running and not env.done:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

        valid_actions = env.get_valid_actions()
        if not valid_actions:
            break

        action = agent.get_action(env.board)
        state, reward, done, _ = env.step(action)
        move_count += 1
        
        draw_game_state(env, move_count)
        time.sleep(delay)

        if env.get_max_tile() >= 2048:
            print("🎉 REACHED 2048! 🎉")
            time.sleep(3)
            break

    # Show final result
    final_font = pygame.font.SysFont('arial', 40, bold=True)
    if env.get_max_tile() >= 2048:
        text = final_font.render("SUCCESS! Reached 2048!", True, (0, 255, 0))
    else:
        text = final_font.render(f"Game Over. Max: {env.get_max_tile()}", True, (255, 0, 0))
    
    text_rect = text.get_rect(center=(WIDTH / 2, HEIGHT / 2))
    screen.blit(text, text_rect)
    pygame.display.update()
    time.sleep(3)

    pygame.quit()
    return env.get_max_tile(), env.score, move_count

# Run visualization
print("Starting visual gameplay...")
max_tile, score, moves = visualize_agent_gameplay(super_agent, delay=0.1)
print(f"Final result: Max tile = {max_tile}, Score = {score}, Moves = {moves}")

Starting visual gameplay...
Final result: Max tile = 256, Score = 5956, Moves = 439


In [ ]:
# Pygame visualization for the fixed agent
import pygame
import time

def visualize_agent_gameplay(agent, delay=0.5):
    """Visualize agent playing with pygame"""
    pygame.init()
    
    # Constants
    SIZE = 4
    TILE_SIZE = 100
    MARGIN = 10
    WIDTH = SIZE * TILE_SIZE + (SIZE + 1) * MARGIN
    HEIGHT = WIDTH + 100  # Extra space for text
    FONT = pygame.font.SysFont("arial", 32)
    INFO_FONT = pygame.font.SysFont("arial", 24)

    # Colors
    BG_COLOR = (187, 173, 160)
    TILE_COLORS = {
        0: (205, 193, 180), 2: (238, 228, 218), 4: (237, 224, 200),
        8: (242, 177, 121), 16: (245, 149, 99), 32: (246, 124, 95),
        64: (246, 94, 59), 128: (237, 207, 114), 256: (237, 204, 97),
        512: (237, 200, 80), 1024: (237, 197, 63), 2048: (237, 194, 46),
        4096: (237, 194, 46)
    }
    TEXT_COLOR_DARK = (119, 110, 101)
    TEXT_COLOR_LIGHT = (249, 246, 242)

    screen = pygame.display.set_mode((WIDTH, HEIGHT))
    pygame.display.set_caption("2048 AI Agent")

    def draw_game_state(env, move_count):
        screen.fill(BG_COLOR)
        
        # Draw board
        for row in range(SIZE):
            for col in range(SIZE):
                value = env.board[row][col]
                color = TILE_COLORS.get(value, (60, 58, 50))
                rect_x = MARGIN + col * (TILE_SIZE + MARGIN)
                rect_y = MARGIN + row * (TILE_SIZE + MARGIN)
                pygame.draw.rect(screen, color, (rect_x, rect_y, TILE_SIZE, TILE_SIZE))
                
                if value != 0:
                    text_color = TEXT_COLOR_DARK if value <= 4 else TEXT_COLOR_LIGHT
                    text = FONT.render(str(value), True, text_color)
                    text_rect = text.get_rect(center=(rect_x + TILE_SIZE / 2, rect_y + TILE_SIZE / 2))
                    screen.blit(text, text_rect)
        
        # Draw info
        info_y = WIDTH + 10
        info_text = INFO_FONT.render(f"Move: {move_count} | Score: {env.score} | Max: {env.get_max_tile()}", True, (0, 0, 0))
        screen.blit(info_text, (10, info_y))
        
        pygame.display.update()

    # Run game
    env = Game2048Env()
    state = env.reset()
    move_count = 0
    running = True

    while running and not env.done:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

        valid_actions = env.get_valid_actions()
        if not valid_actions:
            break

        action = agent.get_action(env.board)
        state, reward, done, _ = env.step(action)
        move_count += 1
        
        draw_game_state(env, move_count)
        time.sleep(delay)

        if env.get_max_tile() >= 2048:
            print("🎉 REACHED 2048! 🎉")
            time.sleep(3)
            break

    # Show final result
    final_font = pygame.font.SysFont('arial', 40, bold=True)
    if env.get_max_tile() >= 2048:
        text = final_font.render("SUCCESS! Reached 2048!", True, (0, 255, 0))
    else:
        text = final_font.render(f"Game Over. Max: {env.get_max_tile()}", True, (255, 0, 0))
    
    text_rect = text.get_rect(center=(WIDTH / 2, HEIGHT / 2))
    screen.blit(text, text_rect)
    pygame.display.update()
    time.sleep(3)

    pygame.quit()
    return env.get_max_tile(), env.score, move_count

# Run visualization
print("Starting visual gameplay...")
max_tile, score, moves = visualize_agent_gameplay(super_agent, delay=0.1)
print(f"Final result: Max tile = {max_tile}, Score = {score}, Moves = {moves}")

Starting visual gameplay...
Final result: Max tile = 256, Score = 5956, Moves = 439


In [82]:
import math
import random
from collections import defaultdict

class MCTSNode:
    def __init__(self, board, parent=None, action=None, is_chance=False):
        self.board = board.copy()
        self.parent = parent
        self.action = action  # Action that led to this node
        self.is_chance = is_chance  # True if this is a chance node (random tile placement)
        
        self.children = []
        self.visits = 0
        self.value_sum = 0.0
        self.untried_actions = None
        
        if not is_chance:
            # Player node - actions are moves
            temp_env = Game2048Env()
            temp_env.board = board.copy()
            self.untried_actions = temp_env.get_valid_actions()
        else:
            # Chance node - actions are tile placements
            empty_positions = list(zip(*np.where(board == 0)))
            self.untried_actions = [(pos, val) for pos in empty_positions for val in [2, 4]]
    
    def is_fully_expanded(self):
        return len(self.untried_actions) == 0
    
    def is_terminal(self):
        if self.is_chance:
            return False
        temp_env = Game2048Env()
        temp_env.board = self.board.copy()
        return temp_env.is_game_over()
    
    def get_average_value(self):
        if self.visits == 0:
            return 0
        return self.value_sum / self.visits

class MCTS2048Agent:
    def __init__(self, iterations=1000, exploration_weight=1.4):
        self.iterations = iterations
        self.exploration_weight = exploration_weight
        self.temp_env = Game2048Env()
        
    def get_action(self, board):
        """Get the best action using MCTS"""
        if np.count_nonzero(board == 0) <= 4:
            # Endgame - use more iterations
            iterations = min(2000, self.iterations * 2)
        else:
            iterations = self.iterations
            
        root = MCTSNode(board, is_chance=False)
        
        for _ in range(iterations):
            # Selection and Expansion
            leaf = self._select_and_expand(root)
            
            # Simulation
            reward = self._simulate(leaf)
            
            # Backpropagation
            self._backpropagate(leaf, reward)
        
        # Choose best action
        if not root.children:
            # Fallback to random valid action
            temp_env = Game2048Env()
            temp_env.board = board.copy()
            valid_actions = temp_env.get_valid_actions()
            return random.choice(valid_actions) if valid_actions else 0
        
        best_child = max(root.children, key=lambda c: c.visits)
        return best_child.action
    
    def _select_and_expand(self, node):
        """Select and expand using UCB1"""
        while not node.is_terminal():
            if not node.is_fully_expanded():
                return self._expand(node)
            else:
                node = self._select_child(node)
        return node
    
    def _expand(self, node):
        """Expand the node by adding a new child"""
        if not node.untried_actions:
            return node
            
        if node.is_chance:
            # Chance node - add tile placement
            (row, col), value = node.untried_actions.pop(0)
            new_board = node.board.copy()
            new_board[row, col] = value
            child = MCTSNode(new_board, parent=node, action=((row, col), value), is_chance=False)
        else:
            # Player node - add move
            action = node.untried_actions.pop(0)
            self.temp_env.board = node.board.copy()
            new_board, _ = self.temp_env._execute_move(node.board, action)
            child = MCTSNode(new_board, parent=node, action=action, is_chance=True)
        
        node.children.append(child)
        return child
    
    def _select_child(self, node):
        """Select child using UCB1 formula"""
        if node.is_chance:
            # For chance nodes, select proportionally to probability
            # Favor 2-tiles (90%) over 4-tiles (10%)
            weights = []
            for child in node.children:
                if child.action[1] == 2:  # 2-tile
                    weights.append(0.9)
                else:  # 4-tile
                    weights.append(0.1)
            
            # Weighted random selection
            return random.choices(node.children, weights=weights)[0]
        else:
            # For player nodes, use UCB1
            return max(node.children, key=lambda c: self._ucb1_value(c, node.visits))
    
    def _ucb1_value(self, child, parent_visits):
        """Calculate UCB1 value for child selection"""
        if child.visits == 0:
            return float('inf')
        
        exploitation = child.get_average_value()
        exploration = self.exploration_weight * math.sqrt(math.log(parent_visits) / child.visits)
        return exploitation + exploration
    
    def _simulate(self, node):
        """Simulate a random rollout from the node"""
        current_board = node.board.copy()
        self.temp_env.board = current_board.copy()
        self.temp_env.score = 0
        self.temp_env.done = False
        
        moves = 0
        max_moves = 100  # Limit simulation length
        
        while not self.temp_env.done and moves < max_moves:
            valid_actions = self.temp_env.get_valid_actions()
            if not valid_actions:
                break
                
            # Use intelligent random policy for simulation
            action = self._intelligent_random_action(self.temp_env.board, valid_actions)
            _, reward, done, _ = self.temp_env.step(action)
            moves += 1
        
        # Return evaluation of final state
        return self._evaluate_board(self.temp_env.board)
    
    def _intelligent_random_action(self, board, valid_actions):
        """Smart random policy that prefers corner-preserving moves"""
        max_tile = np.max(board)
        max_pos = np.unravel_index(np.argmax(board), board.shape)
        
        # Prioritize moves that keep max tile in corner
        if max_pos in [(0,0), (0,3), (3,0), (3,3)]:
            if max_pos == (0,0) and 2 in valid_actions:  # Prefer LEFT
                return 2 if random.random() < 0.7 else random.choice(valid_actions)
            elif max_pos == (0,3) and 3 in valid_actions:  # Prefer RIGHT
                return 3 if random.random() < 0.7 else random.choice(valid_actions)
            elif max_pos == (3,0) and 2 in valid_actions:  # Prefer LEFT
                return 2 if random.random() < 0.7 else random.choice(valid_actions)
            elif max_pos == (3,3) and 3 in valid_actions:  # Prefer RIGHT
                return 3 if random.random() < 0.7 else random.choice(valid_actions)
        
        return random.choice(valid_actions)
    
    def _backpropagate(self, node, reward):
        """Backpropagate the reward up the tree"""
        while node is not None:
            node.visits += 1
            node.value_sum += reward
            node = node.parent
    
    def _evaluate_board(self, board):
        """Enhanced evaluation function for MCTS"""
        if len(self.temp_env.get_valid_actions()) == 0:
            return -100000  # Game over
        
        max_tile = np.max(board)
        empty_tiles = np.count_nonzero(board == 0)
        
        # Massive bonus for reaching 2048!
        if max_tile >= 2048:
            return 1000000 + empty_tiles * 10000
        
        # Strong bonuses for high tiles
        tile_bonus = max_tile * 2
        
        # Corner bonus
        max_pos = np.unravel_index(np.argmax(board), board.shape)
        corner_bonus = 0
        if max_pos in [(0,0), (0,3), (3,0), (3,3)]:
            corner_bonus = max_tile * 10
            if max_pos == (0,0):  # Prefer top-left
                corner_bonus *= 1.5
        
        # Empty space bonus
        empty_bonus = empty_tiles * 100
        
        # Monotonicity bonus
        monotonicity = self._calculate_monotonicity(board)
        
        # Merge potential
        merge_potential = self._calculate_merge_potential(board)
        
        return tile_bonus + corner_bonus + empty_bonus + monotonicity * 50 + merge_potential * 20
    
    def _calculate_monotonicity(self, board):
        """Calculate monotonicity score"""
        # Check for decreasing sequences from top-left
        score = 0
        
        # Horizontal monotonicity
        for row in range(4):
            for col in range(3):
                if board[row, col] != 0 and board[row, col + 1] != 0:
                    if board[row, col] >= board[row, col + 1]:
                        score += 1
        
        # Vertical monotonicity
        for col in range(4):
            for row in range(3):
                if board[row, col] != 0 and board[row + 1, col] != 0:
                    if board[row, col] >= board[row + 1, col]:
                        score += 1
        
        return score
    
    def _calculate_merge_potential(self, board):
        """Calculate merge potential"""
        potential = 0
        
        # Horizontal merges
        for row in range(4):
            for col in range(3):
                if board[row, col] != 0 and board[row, col] == board[row, col + 1]:
                    potential += board[row, col]
        
        # Vertical merges
        for row in range(3):
            for col in range(4):
                if board[row, col] != 0 and board[row, col] == board[row + 1, col]:
                    potential += board[row, col]
        
        return potential

# Hybrid agent that combines MCTS with Expectimax for different situations
class HybridAgent:
    def __init__(self):
        self.mcts_agent = MCTS2048Agent(iterations=1500)
        self.expectimax_agent = UltimateExpectimaxAgent(depth=5)
        
    def get_action(self, board):
        """Choose between MCTS and Expectimax based on game state"""
        max_tile = np.max(board)
        empty_tiles = np.count_nonzero(board == 0)
        
        # Use MCTS for critical endgame situations
        if max_tile >= 512 and empty_tiles <= 6:
            return self.mcts_agent.get_action(board)
        
        # Use MCTS when we have 1024 tile (close to winning)
        if max_tile >= 1024:
            return self.mcts_agent.get_action(board)
        
        # Use Expectimax for early/mid game
        return self.expectimax_agent.get_action(board)

# Enhanced test function with detailed analysis
def detailed_test_runs(agent_class, num_runs=5):
    """Run detailed tests with more analysis"""
    results = []
    
    for run in range(num_runs):
        print(f"\n=== Run {run + 1}/{num_runs} ===")
        
        if agent_class == HybridAgent:
            agent = agent_class()
        else:
            agent = agent_class(iterations=1500) if hasattr(agent_class(), 'iterations') else agent_class()
        
        env = Game2048Env()
        state = env.reset()
        moves = 0
        move_history = []
        
        while not env.done and moves < 3000:  # Increased move limit
            valid_actions = env.get_valid_actions()
            if not valid_actions:
                break
                
            action = agent.get_action(env.board)
            state, reward, done, _ = env.step(action)
            
            move_history.append({
                'move': moves,
                'action': action,
                'max_tile': env.get_max_tile(),
                'empty_tiles': np.count_nonzero(env.board == 0),
                'score': env.score
            })
            
            moves += 1
            
            if moves % 300 == 0:
                print(f"Move {moves}: Max={env.get_max_tile()}, Empty={np.count_nonzero(env.board == 0)}, Score={env.score}")
                
            if env.get_max_tile() >= 2048:
                print(f"🎉 SUCCESS! Reached 2048 in {moves} moves! 🎉")
                break
        
        result = {
            'max_tile': env.get_max_tile(),
            'score': env.score,
            'moves': moves,
            'reached_2048': env.get_max_tile() >= 2048,
            'move_history': move_history
        }
        results.append(result)
        print(f"Result: Max tile = {result['max_tile']}, Score = {result['score']}, Moves = {moves}")
    
    # Detailed analysis
    print(f"\n=== DETAILED ANALYSIS OF {num_runs} RUNS ===")
    max_tiles = [r['max_tile'] for r in results]
    scores = [r['score'] for r in results]
    moves_list = [r['moves'] for r in results]
    success_rate = sum(1 for r in results if r['reached_2048']) / num_runs
    
    print(f"Success rate (2048): {success_rate:.1%}")
    print(f"Average max tile: {np.mean(max_tiles):.0f} (min: {min(max_tiles)}, max: {max(max_tiles)})")
    print(f"Average score: {np.mean(scores):.0f} (min: {min(scores)}, max: {max(scores)})")
    print(f"Average moves: {np.mean(moves_list):.0f} (min: {min(moves_list)}, max: {max(moves_list)})")
    
    # Count how often each max tile was reached
    tile_counts = {}
    for tile in max_tiles:
        tile_counts[tile] = tile_counts.get(tile, 0) + 1
    
    print(f"\nTile distribution:")
    for tile in sorted(tile_counts.keys(), reverse=True):
        print(f"  {tile}: {tile_counts[tile]} times ({tile_counts[tile]/num_runs:.1%})")
    
    return results

# Test different agents
print("=== Testing MCTS Agent ===")
mcts_results = detailed_test_runs(MCTS2048Agent, num_runs=5)

print("\n" + "="*50)
print("=== Testing Hybrid Agent ===")
hybrid_results = detailed_test_runs(HybridAgent, num_runs=5)

=== Testing MCTS Agent ===

=== Run 1/5 ===
Result: Max tile = 32, Score = 212, Moves = 43

=== Run 2/5 ===
Move 300: Max=512, Empty=1, Score=4464
Result: Max tile = 512, Score = 5564, Moves = 385

=== Run 3/5 ===
Result: Max tile = 32, Score = 504, Moves = 76

=== Run 4/5 ===
Move 300: Max=512, Empty=2, Score=4452
Result: Max tile = 512, Score = 4680, Moves = 319

=== Run 5/5 ===
Result: Max tile = 256, Score = 2372, Moves = 202

=== DETAILED ANALYSIS OF 5 RUNS ===
Success rate (2048): 0.0%
Average max tile: 269 (min: 32, max: 512)
Average score: 2666 (min: 212, max: 5564)
Average moves: 205 (min: 43, max: 385)

Tile distribution:
  512: 2 times (40.0%)
  256: 1 times (20.0%)
  32: 2 times (40.0%)

=== Testing Hybrid Agent ===

=== Run 1/5 ===
Move 300: Max=512, Empty=0, Score=4448
Result: Max tile = 512, Score = 5084, Moves = 354

=== Run 2/5 ===
Move 300: Max=512, Empty=4, Score=4496
Result: Max tile = 512, Score = 5284, Moves = 369

=== Run 3/5 ===
Move 300: Max=512, Empty=1, Score

In [83]:
class EnhancedMCTS2048Agent:
    def __init__(self, iterations=3000, exploration_weight=1.4):
        self.iterations = iterations
        self.exploration_weight = exploration_weight
        self.temp_env = Game2048Env()
        
        # Import sophisticated evaluation from UltimateExpectimaxAgent
        self.expectimax_evaluator = UltimateExpectimaxAgent(depth=1)
        
    def get_action(self, board):
        """Enhanced MCTS with better search strategy"""
        empty_tiles = np.count_nonzero(board == 0)
        max_tile = np.max(board)
        
        # Adaptive iterations based on game state
        if max_tile >= 1024:
            iterations = self.iterations * 2  # Critical endgame
        elif empty_tiles <= 6:
            iterations = int(self.iterations * 1.5)  # Late game
        else:
            iterations = self.iterations
            
        root = MCTSNode(board, is_chance=False)
        
        for _ in range(iterations):
            leaf = self._select_and_expand(root)
            reward = self._enhanced_simulate(leaf)
            self._backpropagate(leaf, reward)
        
        if not root.children:
            # Fallback to Expectimax action
            return self.expectimax_evaluator.get_action(board)
        
        # Choose child with best average value (not just most visits)
        best_child = max(root.children, key=lambda c: c.get_average_value() if c.visits > 10 else float('-inf'))
        return best_child.action
    
    def _enhanced_simulate(self, node):
        """Much better simulation using strategic rollout policy"""
        current_board = node.board.copy()
        self.temp_env.board = current_board.copy()
        self.temp_env.score = 0
        self.temp_env.done = False
        
        moves = 0
        max_moves = 50  # Shorter but higher quality rollouts
        
        while not self.temp_env.done and moves < max_moves:
            valid_actions = self.temp_env.get_valid_actions()
            if not valid_actions:
                break
                
            # Use strategic policy instead of random
            action = self._strategic_rollout_policy(self.temp_env.board, valid_actions)
            _, reward, done, _ = self.temp_env.step(action)
            moves += 1
        
        # Use sophisticated evaluation function
        return self.expectimax_evaluator.evaluate(self.temp_env.board)
    
    def _strategic_rollout_policy(self, board, valid_actions):
        """Strategic rollout that uses 2048 domain knowledge"""
        max_tile = np.max(board)
        max_pos = np.unravel_index(np.argmax(board), board.shape)
        empty_tiles = np.count_nonzero(board == 0)
        
        # Priority order based on game state
        action_priorities = {}
        
        # Corner preservation strategy
        if max_pos in [(0,0), (0,3), (3,0), (3,3)]:
            if max_pos == (0,0):  # Top-left corner
                action_priorities = {2: 0.4, 1: 0.3, 0: 0.2, 3: 0.1}  # Prefer LEFT, DOWN
            elif max_pos == (0,3):  # Top-right corner  
                action_priorities = {3: 0.4, 1: 0.3, 0: 0.2, 2: 0.1}  # Prefer RIGHT, DOWN
            elif max_pos == (3,0):  # Bottom-left corner
                action_priorities = {2: 0.4, 0: 0.3, 1: 0.2, 3: 0.1}  # Prefer LEFT, UP
            elif max_pos == (3,3):  # Bottom-right corner
                action_priorities = {3: 0.4, 0: 0.3, 1: 0.2, 2: 0.1}  # Prefer RIGHT, UP
        else:
            # Max tile not in corner - prioritize moving it there
            action_priorities = {2: 0.4, 1: 0.3, 0: 0.2, 3: 0.1}  # Default: prefer LEFT/DOWN
        
        # Filter by valid actions and create weighted choice
        valid_priorities = [(action, action_priorities.get(action, 0.1)) for action in valid_actions]
        
        if not valid_priorities:
            return random.choice(valid_actions)
            
        actions, weights = zip(*valid_priorities)
        
        # Add some randomness but bias toward good moves
        if random.random() < 0.8:  # 80% strategic, 20% random
            return random.choices(actions, weights=weights)[0]
        else:
            return random.choice(valid_actions)

    # Keep existing MCTS tree operations but with minor improvements
    def _select_and_expand(self, node):
        """Enhanced selection with better exploration"""
        path = [node]
        
        while not node.is_terminal():
            if not node.is_fully_expanded():
                return self._expand(node)
            else:
                node = self._enhanced_select_child(node)
                path.append(node)
        return node
    
    def _enhanced_select_child(self, node):
        """Better child selection with domain knowledge"""
        if node.is_chance:
            # For chance nodes, proportional to tile probability
            weights = []
            for child in node.children:
                if child.action[1] == 2:  # 2-tile (90% probability)
                    weights.append(0.9)
                else:  # 4-tile (10% probability)
                    weights.append(0.1)
            return random.choices(node.children, weights=weights)[0]
        else:
            # Enhanced UCB1 with action preferences
            def enhanced_ucb1(child):
                if child.visits == 0:
                    return float('inf')
                
                exploitation = child.get_average_value()
                exploration = self.exploration_weight * math.sqrt(math.log(node.visits) / child.visits)
                
                # Add slight bias toward corner-preserving moves
                action_bias = 0
                if child.action in [2, 1]:  # LEFT, DOWN preferred
                    action_bias = 100
                elif child.action in [0]:  # UP
                    action_bias = 50
                # RIGHT gets no bonus (action 3)
                
                return exploitation + exploration + action_bias
            
            return max(node.children, key=enhanced_ucb1)
    
    def _expand(self, node):
        """Keep existing expand logic"""
        if not node.untried_actions:
            return node
            
        if node.is_chance:
            (row, col), value = node.untried_actions.pop(0)
            new_board = node.board.copy()
            new_board[row, col] = value
            child = MCTSNode(new_board, parent=node, action=((row, col), value), is_chance=False)
        else:
            action = node.untried_actions.pop(0)
            new_board, _ = self.temp_env._execute_move(node.board, action)
            child = MCTSNode(new_board, parent=node, action=action, is_chance=True)
        
        node.children.append(child)
        return child
    
    def _backpropagate(self, node, reward):
        """Keep existing backpropagation"""
        while node is not None:
            node.visits += 1
            node.value_sum += reward
            node = node.parent

# Test the enhanced MCTS
def test_enhanced_mcts():
    print("=== Testing Enhanced MCTS Agent ===")
    agent = EnhancedMCTS2048Agent(iterations=2000)
    
    results = []
    for run in range(3):
        print(f"\n=== Run {run + 1}/3 ===")
        
        env = Game2048Env()
        state = env.reset()
        moves = 0
        
        while not env.done and moves < 2000:
            valid_actions = env.get_valid_actions()
            if not valid_actions:
                break
                
            action = agent.get_action(env.board)
            state, reward, done, _ = env.step(action)
            moves += 1
            
            if moves % 300 == 0:
                print(f"Move {moves}: Max={env.get_max_tile()}, Empty={np.count_nonzero(env.board == 0)}")
                
            if env.get_max_tile() >= 2048:
                print(f"🎉 ENHANCED MCTS SUCCESS! Reached 2048 in {moves} moves! 🎉")
                break
        
        result = {
            'max_tile': env.get_max_tile(),
            'score': env.score, 
            'moves': moves,
            'reached_2048': env.get_max_tile() >= 2048
        }
        results.append(result)
        print(f"Result: Max tile = {result['max_tile']}, Score = {result['score']}")
    
    # Summary
    max_tiles = [r['max_tile'] for r in results]
    success_rate = sum(1 for r in results if r['reached_2048']) / len(results)
    
    print(f"\n=== ENHANCED MCTS SUMMARY ===")
    print(f"Success rate (2048): {success_rate:.1%}")
    print(f"Average max tile: {np.mean(max_tiles):.0f}")
    print(f"Best max tile: {max(max_tiles)}")
    
    return results

# Run the test
enhanced_results = test_enhanced_mcts()

=== Testing Enhanced MCTS Agent ===

=== Run 1/3 ===
Result: Max tile = 128, Score = 1172

=== Run 2/3 ===
Result: Max tile = 64, Score = 500

=== Run 3/3 ===
Result: Max tile = 64, Score = 752

=== ENHANCED MCTS SUMMARY ===
Success rate (2048): 0.0%
Average max tile: 85
Best max tile: 128
